In [18]:
from facebook_business.adobjects.leadgenform import LeadgenForm
from facebook_business.api import FacebookAdsApi
import os
from dotenv import load_dotenv
import requests
import json


In [2]:
load_dotenv()

True

In [3]:
# Initialize the API
FacebookAdsApi.init(access_token=os.getenv('FACEBOOK_ACCESS_TOKEN'),
                    app_id=os.getenv('FACEBOOK_APP_ID'),
                    app_secret=os.getenv('FACEBOOK_APP_SECRET'))

# The ID of the existing lead form you want to copy
existing_lead_form_id = '1778956992538844'

In [4]:
def get_lead_form_data(lead_form_id):
    FacebookAdsApi.init(access_token=os.getenv('FACEBOOK_ACCESS_TOKEN'),
                        app_id=os.getenv('FACEBOOK_APP_ID'),
                        app_secret=os.getenv('FACEBOOK_APP_SECRET'))
    
    lead_form = LeadgenForm(lead_form_id)
    fields = [
        'id',
        'allow_organic_lead',
        'block_display_for_non_targeted_viewer',
        'context_card',
        'created_time',
        'expired_leads_count',
        'follow_up_action_text',
        'follow_up_action_url',
        'is_optimized_for_quality',
        'leads_count',
        'legal_content',
        'locale',
        'name',
        'organic_leads_count',
        'page',
        'page_id',
        'privacy_policy_url',
        'question_page_custom_headline',
        'questions',
        'status',
        'thank_you_page',
        'tracking_parameters',
        # Add any additional fields you need
    ]
    
    lead_form_data = lead_form.api_get(fields=fields)
    
    # Organizing the data into a dictionary
    structured_data = {field: lead_form_data.get(field) for field in fields}
    return structured_data

In [5]:
lead_form_details = get_lead_form_data(existing_lead_form_id)

In [6]:
print(lead_form_details)

{'id': '1778956992538844', 'allow_organic_lead': True, 'block_display_for_non_targeted_viewer': False, 'context_card': <LeadGenContextCard> {
    "content": [
        "Sign up today for your FREE workout and discover why everyone\u2019s talking about \ud83c\udf4atheory!\n\n\u2728 Utilize cutting-edge science to track your goals \n\u2728 Be a part of a supportive community cheers you on every step of the way\n\u2728 Achieve the real and lasting results you\u2019ve always wanted\n\n*Terms and Conditions Apply"
    ],
    "cover_photo": {
        "created_time": "2023-07-25T17:44:57+0000",
        "id": "2018985685120994"
    },
    "id": "668773324702417",
    "style": "PARAGRAPH_STYLE",
    "title": "FREE! FREE! FREE! Try Orangetheory Now!"
}, 'created_time': '2023-07-25T17:45:17+0000', 'expired_leads_count': 0, 'follow_up_action_text': None, 'follow_up_action_url': 'https://www.orangetheory.com/en-us/locations/pennsylvania/coraopolis/630-chauvet-dr/', 'is_optimized_for_quality': False,

In [37]:
# Assume lead_form_details is already populated with the details of the existing form
new_lead_form_data = lead_form_details.copy()

# Update the page_id for the new page
new_lead_form_data['page_id'] = '877867295627373'

# New URL to be used
new_url = "https://www.orangetheory.com/en-us/locations/massachusetts/dedham/610-providence-highway/"

# Update the URLs in the new lead form data
new_lead_form_data['follow_up_action_url'] = new_url
new_lead_form_data['privacy_policy_url'] = new_url


# Remove fields that should not be included in the creation request
new_lead_form_data.pop('id', None)  # Remove the old ID
new_lead_form_data.pop('created_time', None)  # Remove the created time
new_lead_form_data.pop('expired_leads_count', None)  # Remove counts
new_lead_form_data.pop('leads_count', None)
new_lead_form_data.pop('organic_leads_count', None)
new_lead_form_data.pop('page', None)  # Remove old page reference

# Any other fields that are not relevant to creation can also be removed


<Page> {
    "id": "441544299531815",
    "name": "Orangetheory Fitness Robinson -North Fayette"
}

In [38]:
# Define standard question types
standard_question_types = ['FIRST_NAME', 'LAST_NAME', 'EMAIL', 'PHONE', 'CITY', 'STATE', 'COUNTRY', 'STREET_ADDRESS', 'POST_CODE', 'DOB', 'GENDER', 'MARITAL_STATUS', 'JOB_TITLE', 'WORK_PHONE_NUMBER', 'COMPANY_NAME', 'MILITARY_STATUS', 'RELATIONSHIP_STATUS', 'WORK_EMAIL']

# Iterate over the questions and remove the label for standard types
for question in new_lead_form_data['questions']:
    question.pop('id', None)  # Remove 'id' key if it exists
    if question['type'] in standard_question_types and 'label' in question:
        question.pop('label')

# Convert the modified questions list to a JSON string
questions_json_string = json.dumps(new_lead_form_data['questions'])

# Update the questions in your new_lead_form_data dictionary
new_lead_form_data['questions'] = questions_json_string

In [39]:
print(new_lead_form_data)

{'allow_organic_lead': True, 'block_display_for_non_targeted_viewer': False, 'context_card': <LeadGenContextCard> {
    "content": [
        "Sign up today for your FREE workout and discover why everyone\u2019s talking about \ud83c\udf4atheory!\n\n\u2728 Utilize cutting-edge science to track your goals \n\u2728 Be a part of a supportive community cheers you on every step of the way\n\u2728 Achieve the real and lasting results you\u2019ve always wanted\n\n*Terms and Conditions Apply"
    ],
    "cover_photo": {
        "created_time": "2023-07-25T17:44:57+0000",
        "id": "2018985685120994"
    },
    "id": "668773324702417",
    "style": "PARAGRAPH_STYLE",
    "title": "FREE! FREE! FREE! Try Orangetheory Now!"
}, 'follow_up_action_text': None, 'follow_up_action_url': 'https://www.orangetheory.com/en-us/locations/massachusetts/dedham/610-providence-highway/', 'is_optimized_for_quality': False, 'legal_content': <LeadGenLegalContent> {
    "custom_disclaimer": {
        "body": {
    

In [40]:
# Ensure that the environment variable for the access token is set
access_token = os.getenv('FACEBOOK_ACCESS_TOKEN')
if access_token is None:
    print("Access token not found. Please set your environment variable for the access token.")
else:
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    # Facebook API endpoint to create a new lead form
    api_endpoint = f'https://graph.facebook.com/v18.0/{new_lead_form_data["page_id"]}/leadgen_forms'

    # API call to create the new lead form
    response = requests.post(api_endpoint, data=new_lead_form_data, headers=headers)

    # Check response
    if response.status_code == 200:
        print("New lead form created successfully.")
    else:
        print("Failed to create new lead form.")
        print(response.text)


Failed to create new lead form.
{"error":{"message":"Invalid parameter","type":"OAuthException","code":100,"error_subcode":1892075,"is_transient":false,"error_user_title":"Legal content missing","error_user_msg":"Either legal_content_id or privacy_policy is required to be specified to create a LeadGen form","fbtrace_id":"AwZPfh6-eO3gOPTLgiGtH5w"}}
